In [1]:
import utils
import warnings

warnings.filterwarnings('ignore')
utils.set_css_style('style.css')

# 1. Data Splitting

## 1.1. Why split the data?

To use an analogy, let’s say you teach a child to multiply by letting the kid train on the small multiplication table, i.e. everything from 1x1 to 9x9.

Next, you test whether the kid is able to perform the same multiplications. The result is a success. The kid gets it right almost every time.

<img src="figures/multiplication.jpeg" alt="multiplication" style="width: 300px;"/>

What’s the problem here?

You don’t know if the kid understands multiplication at all, or has simply memorized the table!

So what you would do instead is test the kid on multiplications like 11x12, that are outside of the table.

This is exactly why we need to test machine learning models on unseen data. Otherwise, we have no way of knowing whether the algorithm has learned a generalizable pattern or has simply memorized the training data.

## 1.2. Training, validation & test datasets

Data splits usually depends on the use case. But typically, we split data into 3 main datasets:

* Training dataset
* Validation dataset
* Testing dataset

<img src="figures/splits.png" alt="splits" style="width: 500px;"/>

Let's quote the base definitions from Jason Brownlee’s excellent article on this topic, as it is quite comprehensive:

### 1.2.1. Training dataset

> Training Dataset: The sample of data used to fit the model.

The actual dataset that we use to train the model (weights and biases in the case of Neural Network). The model sees and learns from this data.


### 1.2.2. Validation dataset

> Validation Dataset: The sample of data used to provide an unbiased evaluation of a model fit on the training dataset while tuning model hyperparameters. The evaluation becomes more biased as skill on the validation dataset is incorporated into the model configuration.

The validation set is used to evaluate a given model, but this is for frequent evaluation. We as machine learning engineers use this data to fine-tune the model hyperparameters. Hence the model occasionally sees this data, but never does it “Learn” from this. We use the validation set results and update higher level hyperparameters. So the validation set in a way affects a model, but indirectly.

### 1.2.3. Test dataset

> Test Dataset: The sample of data used to provide an unbiased evaluation of a final model fit on the training dataset.

The Test dataset provides the gold standard used to evaluate the model. It is only used once a model is completely trained (using the train and validation sets). The test set is generally what is used to evaluate competing models (For example on many Kaggle competitions, the validation set is released initially along with the training set and the actual test set is only released when the competition is about to close, and it is the result of the the model on the Test set that decides the winner). Many a times the validation set is used as the test set, but it is not good practice. The test set is generally well curated. It contains carefully sampled data that spans the various classes that the model would face, when used in the real world.

### 1.2.4. Dataset split ratio

This mainly depends on 2 things. First, the total number of samples in your data and second, on the actual model you are training.

Some models need substantial data to train upon, so in this case you would optimize for the larger training sets. Models with very few hyperparameters will be easy to validate and tune, so you can probably reduce the size of your validation set, but if your model has many hyperparameters, you would want to have a large validation set as well(although you should also consider cross validation). Also, if you happen to have a model with no hyperparameters or ones that cannot be easily tuned, you probably don’t need a validation set too!

Like many other things in machine learning, the train-test-validation split ratio is also quite specific to your use case and it gets easier to make judge ment as you train and build more and more models.


## 1.3. Cross-validation

When there is never enough data to train your model, removing a part of it for validation poses a problem of underfitting. By reducing the training data, we risk losing important patterns/ trends in data set, which in turn increases error induced by bias. So, what we require is a method that provides ample data for training the model and also leaves ample data for validation. Cross validation does exactly that.

### 1.3.1. K-fold cross validation

In K Fold cross validation, the data is divided into k subsets. Now the holdout method is repeated k times, such that each time, one of the k subsets is used as the test set/ validation set and the other k-1 subsets are put together to form a training set. The error estimation is averaged over all k trials to get total effectiveness of our model. As can be seen, every data point gets to be in a validation set exactly once, and gets to be in a training set k-1 times. This significantly reduces bias as we are using most of the data for fitting, and also significantly reduces variance as most of the data is also being used in validation set. Interchanging the training and test sets also adds to the effectiveness of this method. As a general rule and empirical evidence, K = 5 or 10 is generally preferred, but nothing’s fixed and it can take any value.

<img src="figures/cross-validation.png" alt="cross-validation" style="width: 500px;"/>


### 1.3.2. Stratified k-fold cross validation

In some cases, there may be a large imbalance in the response variables. For example, in dataset concerning price of houses, there might be large number of houses having high price. Or in case of classification, there might be several times more negative samples than positive samples. For such problems, a slight variation in the K Fold cross validation technique is made, such that each fold contains approximately the same percentage of samples of each target class as the complete set, or in case of regression problems, the mean response value is approximately equal in all the folds. This variation is also known as Stratified K Fold.

These validation techniques are also referred to as Non-exhaustive cross validation methods. These do not compute all ways of splitting the original sample. Exhaustive Methods, on the other hand, compute all possible ways the data can be split into training and test sets.

### 1.3.3. Leave-P-Out cross validation

This approach leaves p data points out of training data, i.e. if there are n data points in the original sample then, n-p samples are used to train the model and p points are used as the validation set. This is repeated for all combinations in which original sample can be separated this way, and then the error is averaged for all trials, to give overall effectiveness.

This method is exhaustive in the sense that it needs to train and validate the model for all possible combinations, and for moderately large p, it can become computationally infeasible.

## 1.4. Splitting data with sklearn

Let’s see how to do split a dataset into a training and a testing datasets in Python. We’ll do this using the `scikit-learn` library and specifically the `train_test_split` method. We’ll start with importing the necessary libraries:

In [2]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split

Let’s load in the diabetes dataset, turn it into a data frame and define the columns’ names:

In [3]:
# Load the Diabetes dataset
columns = ["age","sex","bmi","map","tc","ldl","hdl","tch","ltg","glu"] # columns names
diabetes = datasets.load_diabetes() # Call the diabetes dataset from sklearn
df = pd.DataFrame(diabetes.data, columns=columns) # load the dataset as a pandas data frame
y = diabetes.target # define the target variable (dependent variable) as y
print("Dataset shape: ", df.shape, y.shape)

Dataset shape:  (442, 10) (442,)


In [4]:
print(diabetes.DESCR)

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, T-Cells (a type of white blood cells)
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, thyroid stimulating hormone
      - s5      ltg, lamotrigine
      - s6      glu, blood sugar level

Note: Each of these 10 feature va

Let's visualize our features table:

In [5]:
df.head()

,age,sex,bmi,map,tc,ldl,hdl,tch,ltg,glu
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641


And our target array:

In [6]:
y[1:5]

array([ 75., 141., 206., 135.])

Now we can use the `train_test_split` function in order to make the split. The `test_size=0.2` inside the function indicates the percentage of the data that should be held over for testing. It’s usually around 80/20 or 70/30.

In [7]:
# create training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2)
print("Training set: ", X_train.shape, y_train.shape)
print("Testing set: ", X_test.shape, y_test.shape)

Training set:  (353, 10) (353,)
Testing set:  (89, 10) (89,)
